In [ ]:
# Asset Classes Based Portfolio Optimization and Analyses

import pandas as pd
import numpy as np
from scipy.optimize import minimize
import statsmodels.api as sm
from scipy.stats import skew, kurtosis, ttest_rel, f_oneway, wilcoxon, ttest_1samp, ttest_ind, shapiro, t
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from matplotlib.dates import DateFormatter, YearLocator
import warnings
from sklearn.covariance import LedoitWolf
import uuid

In [ ]:
# Optional imports
try:
    from pmdarima import auto_arima
    PMDARIMA_AVAILABLE = True
except ImportError:
    PMDARIMA_AVAILABLE = False
    print("pmdarima not installed. Skipping Auto-ARIMA model.")

try:
    from xgboost import XGBRegressor
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("xgboost not installed. Skipping XGBoost model.")

try:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense
    from sklearn.preprocessing import MinMaxScaler
    TENSORFLOW_AVAILABLE = True
except ImportError:
    TENSORFLOW_AVAILABLE = False
    print("tensorflow not installed. Skipping LSTM model.")

warnings.filterwarnings("ignore")

In [ ]:
# Load and preprocess data
file_path = "C:/Users/IMI/Documents/Courses/SAPM/AY 2025-26/R-Exercises/portfolio_data_extended.csv"
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    raise FileNotFoundError(f"Data file not found at {file_path}")

df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
print(f"Removed {df['Date'].isna().sum()} rows with invalid dates")
df = df.dropna(subset=['Date'])
print(f"Removed {df.index.duplicated().sum()} duplicate dates")
df = df[~df['Date'].duplicated(keep='first')]
df.set_index('Date', inplace=True)
df = df.sort_index()

In [ ]:
# Validate data
if 'Sensex' not in df.columns:
    raise ValueError("Column 'Sensex' not found in the dataset")
if (df <= 0).any().any():
    print("Warning: Non-positive values found in data, replacing with forward fill")
    df = df.where(df > 0).fillna(method='ffill')

if (~np.isfinite(df)).any().any():
    print("Warning: Non-finite values in data, replacing with forward fill")
    df = df.where(np.isfinite(df)).fillna(method='ffill')

In [ ]:
# Diagnostics
print("\nData Diagnostics:")
print("Is index monotonic increasing?", df.index.is_monotonic_increasing)
print("Number of duplicated dates:", df.index.duplicated().sum())
print("Any NaT values in index?", df.index.isna().sum())
print("First 10 dates:", df.index[:10])
print("Last 10 dates:", df.index[-10:])
print("Length of df:", len(df))

In [ ]:
# Calculate log returns
log_returns = np.log(df / df.shift(1)).dropna()
print("\nLog Returns Diagnostics:")
print("Is log_returns index monotonic increasing?", log_returns.index.is_monotonic_increasing)
print("Number of duplicated dates in log_returns:", log_returns.index.duplicated().sum())
print("Any NaT values in log_returns index?", log_returns.index.isna().sum())
print("Any non-finite values in log_returns:", (~np.isfinite(log_returns)).sum().sum())
if (~np.isfinite(log_returns)).any().any():
    print("Warning: Non-finite values in log_returns, replacing with 0")
    log_returns = log_returns.where(np.isfinite(log_returns), 0)


In [ ]:
# Define market and asset returns
market_returns = log_returns['Sensex']
log_returns_subset = log_returns.drop(columns=['Sensex'], errors='ignore')
num_assets = len(log_returns_subset.columns)

# Annualize covariance matrix with Ledoit-Wolf shrinkage
cov_matrix = LedoitWolf().fit(log_returns).covariance_ * 252
port_cov_matrix = LedoitWolf().fit(log_returns_subset).covariance_ * 252
market_var = cov_matrix[log_returns.columns.get_loc('Sensex'), log_returns.columns.get_loc('Sensex')]
asset_market_cov = cov_matrix[log_returns_subset.columns.get_indexer(log_returns_subset.columns), log_returns.columns.get_loc('Sensex')]

# Check covariance matrix
cond_number = np.linalg.cond(port_cov_matrix)
print(f"Covariance matrix condition number: {cond_number:.2e}")
if cond_number > 1e6:
    print("Warning: Covariance matrix is ill-conditioned, applying regularization")
    port_cov_matrix += np.eye(port_cov_matrix.shape[0]) * 1e-4

# Set risk-free rate and market return
risk_free_rate = 0.0696
market_return = market_returns.mean() * 252
if not np.isfinite(market_return):
    print("Warning: Non-finite market return, setting to 0")
    market_return = 0

# Asset class mapping
asset_class_groups = {
    'Equities': [
        'Reliance Industries', 'HDFC Bank', 'Tata Consultancy Services', 'ICICI Bank', 'Infosys',
        'Bharti Airtel', 'State Bank of India', 'Hindustan Unilever', 'ITC', 'Kotak Mahindra Bank',
        'Larsen & Toubro', 'Axis Bank', 'Asian Paints', 'Maruti Suzuki India', 'Sun Pharmaceutical Industries',
        'Bajaj Finance', 'Nestle India', 'HCL Technologies', 'Mahindra & Mahindra', 'Titan Company',
        'UltraTech Cement', 'Power Grid Corporation of India', 'NTPC', 'Tata Motors', 'IndusInd Bank',
        'Bajaj Auto', 'Tech Mahindra', 'JSW Steel', 'Wipro', 'Adani Enterprises'
    ],
    'Forex': ['USD/INR', 'EUR/INR'],
    'Gold': ['Gold BeES']
}

target_exposures = {
    'Equities': 0.7,
    'Forex': 0.15,
    'Gold': 0.15
}

# Validate asset names
all_assets = set(log_returns_subset.columns)
for asset_class, assets in asset_class_groups.items():
    invalid_assets = [asset for asset in assets if asset not in all_assets]
    if invalid_assets:
        print(f"Warning: Invalid asset names in {asset_class}: {invalid_assets}")

asset_class_indices = {}
for asset_class, assets in asset_class_groups.items():
    indices = [log_returns_subset.columns.get_loc(asset) for asset in assets if asset in log_returns_subset.columns]
    asset_class_indices[asset_class] = indices
print("\nAsset Class Indices:", asset_class_indices)


In [ ]:
# Helper functions
def calc_standard_deviation(weights, cov_matrix):
    try:
        variance = weights.T @ cov_matrix @ weights
        if not np.isfinite(variance) or variance <= 0:
            return np.nan
        return np.sqrt(variance)
    except Exception as e:
        print(f"Error in calc_standard_deviation: {e}")
        return np.nan

def calc_expected_return(weights, returns):
    try:
        ret = np.sum(returns.mean() * weights) * 252
        return ret if np.isfinite(ret) else np.nan
    except Exception as e:
        print(f"Error in calc_expected_return: {e}")
        return np.nan

def calc_sharpe_ratio(weights, returns, cov_matrix, rf_rate):
    try:
        port_return = calc_expected_return(weights, returns)
        port_std = calc_standard_deviation(weights, cov_matrix)
        if not np.isfinite(port_return) or not np.isfinite(port_std) or port_std == 0:
            return np.nan
        return (port_return - rf_rate) / port_std
    except Exception as e:
        print(f"Error in calc_sharpe_ratio: {e}")
        return np.nan

def calc_beta(weights, asset_market_cov, market_var):
    try:
        port_market_cov = weights @ asset_market_cov
        if not np.isfinite(port_market_cov) or market_var == 0:
            return np.nan
        return port_market_cov / market_var
    except Exception as e:
        print(f"Error in calc_beta: {e}")
        return np.nan

def calc_treynor_ratio(weights, returns, cov_matrix, rf_rate, asset_market_cov, market_var):
    try:
        port_return = calc_expected_return(weights, returns)
        port_beta = calc_beta(weights, asset_market_cov, market_var)
        if not np.isfinite(port_return) or not np.isfinite(port_beta) or port_beta == 0:
            return np.nan
        return (port_return - rf_rate) / port_beta
    except Exception as e:
        print(f"Error in calc_treynor_ratio: {e}")
        return np.nan

def calc_jensens_alpha(weights, returns, cov_matrix, rf_rate, mkt_return, asset_market_cov, market_var):
    try:
        port_return = calc_expected_return(weights, returns)
        port_beta = calc_beta(weights, asset_market_cov, market_var)
        if not np.isfinite(port_return) or not np.isfinite(port_beta):
            return np.nan
        return port_return - (rf_rate + port_beta * (mkt_return - rf_rate))
    except Exception as e:
        print(f"Error in calc_jensens_alpha: {e}")
        return np.nan

def calc_var(weights, returns, alpha=0.05):
    try:
        port_returns = returns @ weights
        if not np.isfinite(port_returns).all():
            return np.nan
        return np.percentile(port_returns, 100 * alpha)
    except Exception as e:
        print(f"Error in calc_var: {e}")
        return np.nan

def calc_cvar(weights, returns, alpha=0.05):
    try:
        port_returns = returns @ weights
        if not np.isfinite(port_returns).all():
            return np.nan
        var = calc_var(weights, returns, alpha)
        if np.isnan(var):
            return np.nan
        cvar = port_returns[port_returns <= var].mean()
        return cvar if np.isfinite(cvar) else np.nan
    except Exception as e:
        print(f"Error in calc_cvar: {e}")
        return np.nan


In [ ]:
# Objective functions
def calc_neg_sharpe_ratio(weights, returns, cov_matrix, rf_rate, asset_market_cov, market_var):
    try:
        sharpe = calc_sharpe_ratio(weights, returns, cov_matrix, rf_rate)
        return -sharpe if np.isfinite(sharpe) else np.inf
    except Exception as e:
        print(f"Error in calc_neg_sharpe_ratio: {e}")
        return np.inf

def calc_variance(weights, returns, cov_matrix, rf_rate, asset_market_cov, market_var):
    try:
        variance = weights.T @ cov_matrix @ weights
        return variance if np.isfinite(variance) else np.inf
    except Exception as e:
        print(f"Error in calc_variance: {e}")
        return np.inf

def neg_var(weights, returns, cov_matrix, rf_rate, asset_market_cov, market_var):
    try:
        var = calc_var(weights, returns)
        return -var if np.isfinite(var) else np.inf
    except Exception as e:
        print(f"Error in neg_var: {e}")
        return np.inf

def neg_cvar(weights, returns, cov_matrix, rf_rate, asset_market_cov, market_var):
    try:
        cvar = calc_cvar(weights, returns)
        return -cvar if np.isfinite(cvar) else np.inf
    except Exception as e:
        print(f"Error in neg_cvar: {e}")
        return np.inf

def calc_neg_treynor_ratio(weights, returns, cov_matrix, rf_rate, asset_market_cov, market_var):
    try:
        treynor = calc_treynor_ratio(weights, returns, cov_matrix, rf_rate, asset_market_cov, market_var)
        return -treynor if np.isfinite(treynor) else np.inf
    except Exception as e:
        print(f"Error in calc_neg_treynor_ratio: {e}")
        return np.inf

objective_dict = {
    'max_sharpe': calc_neg_sharpe_ratio,
    'min_variance': calc_variance,
    'min_VaR': neg_var,
    'min_CVaR': neg_cvar,
    'max_treynor': calc_neg_treynor_ratio,
}


In [ ]:
# Performance Metrics
def compute_performance_metrics(weights, returns, cov_matrix, rf_rate, mkt_returns, mkt_return, asset_market_cov, market_var):
    try:
        port_returns = returns @ weights
        if not np.isfinite(port_returns).all():
            print("Warning: Non-finite values in port_returns")
            port_returns = port_returns.where(np.isfinite(port_returns), 0)
        ann_return = calc_expected_return(weights, returns)
        ann_var = weights.T @ cov_matrix @ weights
        ann_std = np.sqrt(ann_var) if np.isfinite(ann_var) and ann_var > 0 else np.nan
        winning_ratio = (port_returns > 0).mean() if len(port_returns) > 0 else np.nan
        port_skew = skew(port_returns, nan_policy='omit') if len(port_returns) > 1 else np.nan
        port_kurt = kurtosis(port_returns, nan_policy='omit') if len(port_returns) > 1 else np.nan
        var_5 = np.percentile(port_returns, 5) if len(port_returns) > 0 and np.isfinite(port_returns).all() else np.nan
        cvar_5 = port_returns[port_returns <= var_5].mean() if len(port_returns[port_returns <= var_5]) > 0 else np.nan
        tail_ratio = cvar_5 / var_5 if np.isfinite(var_5) and var_5 != 0 else np.nan
        port_beta = calc_beta(weights, asset_market_cov, market_var)
        port_alpha = calc_jensens_alpha(weights, returns, cov_matrix, rf_rate, mkt_return, asset_market_cov, market_var)
        ols = sm.OLS(port_returns, sm.add_constant(mkt_returns)).fit()
        r_squared = ols.rsquared if np.isfinite(ols.rsquared) else np.nan
        port_sharpe = calc_sharpe_ratio(weights, returns, cov_matrix, rf_rate)
        port_treynor = calc_treynor_ratio(weights, returns, cov_matrix, rf_rate, asset_market_cov, market_var)
        tracking_error = np.std(port_returns - mkt_returns) if len(port_returns) > 0 else np.nan
        info_ratio = (ann_return - mkt_return) / tracking_error if np.isfinite(tracking_error) and tracking_error != 0 else np.nan
        downside_returns = port_returns[port_returns < 0]
        sortino = (ann_return - rf_rate) / np.sqrt(np.mean(downside_returns**2)) if len(downside_returns) > 0 and np.isfinite(np.mean(downside_returns**2)) else np.nan
        common_sense = ann_return / ann_std if np.isfinite(ann_std) and ann_std != 0 else np.nan
        cum_returns = np.exp(np.cumsum(port_returns)) - 1
        rolling_max = np.maximum.accumulate(cum_returns + 1)
        drawdowns = (rolling_max - (cum_returns + 1)) / rolling_max
        max_drawdown = drawdowns.max() if len(drawdowns) > 0 and np.isfinite(drawdowns).all() else np.nan
        calmar = ann_return / max_drawdown if np.isfinite(max_drawdown) and max_drawdown != 0 else np.nan
        gains = port_returns[port_returns > 0].sum() if len(port_returns[port_returns > 0]) > 0 else 0
        losses = -port_returns[port_returns < 0].sum() if len(port_returns[port_returns < 0]) > 0 else 0
        omega = gains / losses if losses != 0 else np.nan

        hurst = np.nan
        if len(port_returns) >= 16:
            try:
                min_block = 8
                max_block = len(port_returns) // 2
                block_sizes = np.unique(np.logspace(np.log10(min_block), np.log10(max_block), num=10, dtype=int))
                rs_values = []
                for n in block_sizes:
                    num_blocks = len(port_returns) // n
                    if num_blocks < 2:
                        continue
                    rs_block = []
                    for i in range(num_blocks):
                        block = port_returns[i*n:(i+1)*n]
                        if len(block) < 2 or np.std(block, ddof=1) == 0:
                            continue
                        mean = np.mean(block)
                        y = block - mean
                        z = np.cumsum(y)
                        r = np.max(z) - np.min(z)
                        s = np.std(block, ddof=1)
                        if s > 0:
                            rs_block.append(r / s)
                    rs_values.append(np.mean(rs_block) if rs_block else np.nan)
                block_sizes = block_sizes[:len(rs_values)]
                rs_values = np.array(rs_values)
                valid_idx = ~np.isnan(rs_values) & (rs_values > 0)
                if sum(valid_idx) >= 2:
                    hurst = np.polyfit(np.log(block_sizes[valid_idx]), np.log(rs_values[valid_idx]), 1)[0]
                    print(f"R/S Hurst exponent valid points: {sum(valid_idx)}")
                else:
                    print("Warning: Not enough valid R/S points for Hurst exponent.")
            except Exception as e:
                print(f"Error in Hurst exponent calculation: {e}")

        downside_dev = np.sqrt(np.mean(np.minimum(port_returns - rf_rate / 252, 0)**2)) if len(port_returns) > 0 else np.nan

        metrics = {
            'Annualized Return': ann_return,
            'Annualized Variance': ann_var,
            'Annualized Std Dev': ann_std,
            'Winning Day Ratio': winning_ratio,
            'Skewness': port_skew,
            'Kurtosis': port_kurt,
            'VaR (5%)': var_5,
            'CVaR (5%)': cvar_5,
            'Tail Ratio': tail_ratio,
            'Alpha': port_alpha,
            'Beta': port_beta,
            'R-Squared': r_squared,
            'Sharpe Ratio': port_sharpe,
            'Treynor Ratio': port_treynor,
            'Tracking Error': tracking_error,
            'Information Ratio': info_ratio,
            'Sortino Ratio': sortino,
            'Common Sense Ratio': common_sense,
            'Max Drawdown': max_drawdown,
            'Calmar Ratio': calmar,
            'Omega Ratio': omega,
            'Hurst Exponent': hurst,
            'Downside Deviation': downside_dev
        }
        return metrics, drawdowns
    except Exception as e:
        print(f"Error in compute_performance_metrics: {e}")
        return {}, np.array([])



In [ ]:
# Compute baseline metrics
initial_weights = np.array([1 / num_assets] * num_assets)
for asset_class, target in target_exposures.items():
    indices = asset_class_indices.get(asset_class, [])
    if indices:
        current_sum = sum(initial_weights[i] for i in indices)
        if current_sum != 0:
            scaling_factor = target / current_sum
            for i in indices:
                initial_weights[i] *= scaling_factor
initial_weights = initial_weights / np.sum(initial_weights)
np.random.seed(42)
initial_weights_perturbed = np.random.uniform(0, 0.1, num_assets)
for asset_class, target in target_exposures.items():
    indices = asset_class_indices.get(asset_class, [])
    if indices:
        current_sum = sum(initial_weights_perturbed[i] for i in indices)
        if current_sum != 0:
            scaling_factor = target / current_sum
            for i in indices:
                initial_weights_perturbed[i] *= scaling_factor
initial_weights_perturbed = initial_weights_perturbed / np.sum(initial_weights_perturbed)
equal_weights = initial_weights
equal_metrics, equal_drawdowns = compute_performance_metrics(equal_weights, log_returns_subset, port_cov_matrix, risk_free_rate, market_returns, market_return, asset_market_cov, market_var)
print("\nEqual-Weight Portfolio Metrics (Baseline):")
for k, v in equal_metrics.items():
    print(f"{k}: {v:.4f}")

equal_vol = equal_metrics.get('Annualized Std Dev', np.nan)
equal_beta = equal_metrics.get('Beta', np.nan)
equal_var = equal_metrics.get('VaR (5%)', np.nan)
equal_cvar = equal_metrics.get('CVaR (5%)', np.nan)
equal_treynor = equal_metrics.get('Treynor Ratio', np.nan)

safe_equal_vol = equal_vol if np.isfinite(equal_vol) else 0.2
safe_equal_beta = equal_beta if np.isfinite(equal_beta) else 1.0
safe_equal_var = equal_var if np.isfinite(equal_var) else -0.06
safe_equal_cvar = equal_cvar if np.isfinite(equal_cvar) else -0.09
safe_equal_treynor = equal_treynor if np.isfinite(equal_treynor) else 0.03


In [ ]:
# Define constraints
def safe_vol_constraint(weights, max_vol=safe_equal_vol):
    vol = calc_standard_deviation(weights, port_cov_matrix)
    return max_vol - vol if np.isfinite(vol) else -1e10

def safe_beta_lower_constraint(weights, min_beta=0.65):
    beta = calc_beta(weights, asset_market_cov, market_var)
    return beta - min_beta if np.isfinite(beta) else -1e10

def safe_beta_upper_constraint(weights, max_beta=1.35):
    beta = calc_beta(weights, asset_market_cov, market_var)
    return max_beta - beta if np.isfinite(beta) else -1e10

def safe_var_constraint(weights, min_var=-0.06):
    var = calc_var(weights, log_returns_subset)
    return -min_var - var if np.isfinite(var) else -1e10

def safe_cvar_constraint(weights, min_cvar=-0.09):
    cvar = calc_cvar(weights, log_returns_subset)
    return -min_cvar - cvar if np.isfinite(cvar) else -1e10

def safe_treynor_constraint(weights, min_treynor=0.03):
    treynor = calc_treynor_ratio(weights, log_returns_subset, port_cov_matrix, risk_free_rate, asset_market_cov, market_var)
    return treynor - min_treynor if np.isfinite(treynor) else -1e10

def safe_asset_class_constraint(weights, positions, target_weight):
    return sum(weights[i] for i in positions) - target_weight

constraint_sets = {
    'constraint1': [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
    ],
    'constraint2': [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
        {'type': 'ineq', 'fun': safe_vol_constraint},
    ],
    'constraint3': [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
        {'type': 'ineq', 'fun': safe_vol_constraint},
        {'type': 'ineq', 'fun': safe_beta_lower_constraint},
        {'type': 'ineq', 'fun': safe_beta_upper_constraint},
    ],
    'constraint4': [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
        {'type': 'ineq', 'fun': safe_vol_constraint},
        {'type': 'ineq', 'fun': safe_beta_lower_constraint},
        {'type': 'ineq', 'fun': safe_beta_upper_constraint},
    ],
    'constraint5': [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
        {'type': 'ineq', 'fun': safe_vol_constraint},
        {'type': 'ineq', 'fun': safe_var_constraint},
    ],
    'constraint6': [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
        {'type': 'ineq', 'fun': safe_vol_constraint},
        {'type': 'ineq', 'fun': safe_cvar_constraint},
    ],
    'constraint7': [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
        {'type': 'ineq', 'fun': safe_vol_constraint},
        {'type': 'ineq', 'fun': safe_treynor_constraint},
    ],
    'constraint8': [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
        {'type': 'ineq', 'fun': safe_vol_constraint},
        {'type': 'ineq', 'fun': safe_beta_lower_constraint},
        {'type': 'ineq', 'fun': safe_beta_upper_constraint},
        {'type': 'ineq', 'fun': safe_var_constraint},
        {'type': 'ineq', 'fun': safe_cvar_constraint},
        {'type': 'ineq', 'fun': safe_treynor_constraint},
    ],
}

for set_name in constraint_sets:
    for asset_class, positions in asset_class_indices.items():
        target_weight = target_exposures[asset_class]
        constraint_sets[set_name].append(
            {'type': 'eq', 'fun': lambda w, pos=positions, tw=target_weight: safe_asset_class_constraint(w, pos, tw)}
        )


In [ ]:
# Optimization
bounds = [(0.0, 0.2) for _ in range(num_assets)]
selected_objective = 'max_sharpe'  # Options: 'max_sharpe', 'min_variance', 'min_VaR', 'min_CVaR', 'max_treynor'
selected_constraint = 'constraint8'
objective_fun = objective_dict.get(selected_objective, calc_neg_sharpe_ratio)
constraints = constraint_sets.get(selected_constraint, constraint_sets['constraint1'])
print(f"\nOptimizing for {selected_objective} with {selected_constraint}")

try:
    optimized_results = minimize(
        objective_fun,
        initial_weights_perturbed,
        args=(log_returns_subset, port_cov_matrix, risk_free_rate, asset_market_cov, market_var),
        method='SLSQP',
        constraints=constraints,
        bounds=bounds,
        options={'disp': True, 'maxiter': 2000, 'ftol': 1e-9, 'eps': 1e-8}
    )
    if not optimized_results.success:
        print(f"Warning: Optimization for {selected_objective} with {selected_constraint} failed: {optimized_results.message}")
        print("Falling back to constraint1")
        constraints = constraint_sets['constraint1']
        optimized_results = minimize(
            objective_fun,
            initial_weights_perturbed,
            args=(log_returns_subset, port_cov_matrix, risk_free_rate, asset_market_cov, market_var),
            method='SLSQP',
            constraints=constraints,
            bounds=bounds,
            options={'disp': True, 'maxiter': 2000, 'ftol': 1e-9, 'eps': 1e-8}
        )
        if not optimized_results.success:
            print(f"Warning: Fallback optimization with constraint1 failed: {optimized_results.message}")
            optimal_weights = initial_weights
        else:
            optimal_weights = optimized_results.x
            print(f"Fallback optimization with constraint1 successful, objective value: {optimized_results.fun:.4f}")
    else:
        optimal_weights = optimized_results.x
        print(f"Optimization for {selected_objective} with {selected_constraint} successful, objective value: {optimized_results.fun:.4f}")
except Exception as e:
    print(f"Error in optimization for {selected_objective} with {selected_constraint}: {e}")
    optimal_weights = initial_weights

# Verify constraints
print("\nVerifying Optimal Portfolio Constraints:")
try:
    opt_vol = calc_standard_deviation(optimal_weights, port_cov_matrix)
    opt_beta = calc_beta(optimal_weights, asset_market_cov, market_var)
    opt_var = calc_var(optimal_weights, log_returns_subset)
    opt_cvar = calc_cvar(optimal_weights, log_returns_subset)
    opt_treynor = calc_treynor_ratio(optimal_weights, log_returns_subset, port_cov_matrix, risk_free_rate, asset_market_cov, market_var)
    print(f"Optimal volatility: {opt_vol:.4f} (should be <= {safe_equal_vol:.4f})")
    print(f"Optimal VaR (5%): {opt_var:.4f} (should be >= -0.06 for constraint5, constraint8)")
    print(f"Optimal CVaR (5%): {opt_cvar:.4f} (should be >= -0.09 for constraint6, constraint8)")
    print(f"Optimal Treynor Ratio: {opt_treynor:.4f} (should be >= 0.03 for constraint7, constraint8)")
    for asset_class, positions in asset_class_indices.items():
        asset_class_weight = sum(optimal_weights[i] for i in positions)
        target_weight = target_exposures[asset_class]
        print(f"{asset_class} weight: {asset_class_weight:.4f} (should be = {target_weight:.4f})")
except Exception as e:
    print(f"Error in verifying constraints: {e}")

# Optimal portfolio metrics
try:
    opt_return = calc_expected_return(optimal_weights, log_returns_subset)
    opt_vol = calc_standard_deviation(optimal_weights, port_cov_matrix)
    opt_sharpe = calc_sharpe_ratio(optimal_weights, log_returns_subset, port_cov_matrix, risk_free_rate)
    opt_beta = calc_beta(optimal_weights, asset_market_cov, market_var)
    opt_treynor = calc_treynor_ratio(optimal_weights, log_returns_subset, port_cov_matrix, risk_free_rate, asset_market_cov, market_var)
    opt_alpha = calc_jensens_alpha(optimal_weights, log_returns_subset, port_cov_matrix, risk_free_rate, market_return, asset_market_cov, market_var)

    print("\nOptimal Weights:")
    for ticker, weight in zip(log_returns_subset.columns, optimal_weights):
        print(f"{ticker}: {weight:.4f}")
    print(f"Expected Annual Return: {opt_return:.4f}")
    print(f"Expected Volatility: {opt_vol:.4f}")
    print(f"Sharpe Ratio: {opt_sharpe:.4f}")
    print(f"Treynor Ratio: {opt_treynor:.4f}")
    print(f"Jensen's Alpha: {opt_alpha:.4f}")
except Exception as e:
    print(f"Error in computing optimal portfolio metrics: {e}")

optimal_metrics, optimal_drawdowns = compute_performance_metrics(
    optimal_weights, log_returns_subset, port_cov_matrix, risk_free_rate,
    market_returns, market_return, asset_market_cov, market_var
)
print("\nOptimal Portfolio Metrics:")
for k, v in optimal_metrics.items():
    print(f"{k}: {v:.4f}")


In [ ]:
# Sensitivity analysis for risk-free rate (only for max_sharpe and max_treynor)
sensitivity_results = []
if selected_objective in ['max_sharpe', 'max_treynor']:
    risk_free_rates = [0.04, 0.0696, 0.10]
    print("\nPerforming Risk-Free Rate Sensitivity Analysis (for max_sharpe or max_treynor):")
    for rf in risk_free_rates:
        try:
            opt_res = minimize(
                objective_fun,
                initial_weights_perturbed,
                args=(log_returns_subset, port_cov_matrix, rf, asset_market_cov, market_var),
                method='SLSQP',
                constraints=constraints,
                bounds=bounds,
                options={'disp': True, 'maxiter': 2000, 'ftol': 1e-9, 'eps': 1e-8}
            )
            if opt_res.success:
                opt_return = calc_expected_return(opt_res.x, log_returns_subset)
                opt_vol = calc_standard_deviation(opt_res.x, port_cov_matrix)
                opt_beta = calc_beta(opt_res.x, asset_market_cov, market_var)
                opt_var = calc_var(opt_res.x, log_returns_subset)
                opt_cvar = calc_cvar(opt_res.x, log_returns_subset)
                opt_treynor = calc_treynor_ratio(opt_res.x, log_returns_subset, port_cov_matrix, rf, asset_market_cov, market_var)
                sensitivity_results.append((rf, opt_return, opt_vol, opt_beta, opt_var, opt_cvar, opt_treynor))
                print(f"Risk-Free Rate {rf:.4f} optimization successful, objective value: {opt_res.fun:.4f}")
            else:
                print(f"Warning: Sensitivity analysis for risk-free rate {rf:.4f} failed: {opt_res.message}")
                sensitivity_results.append((rf, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan))
        except Exception as e:
            print(f"Error in sensitivity analysis for risk-free rate {rf:.4f}: {e}")
            sensitivity_results.append((rf, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan))
    print("\nRisk-Free Rate Sensitivity Analysis Results:")
    for rf, ret, vol, beta, var, cvar, treynor in sensitivity_results:
        print(f"Risk-Free Rate: {rf:.4f}, Return: {ret:.4f}, Volatility: {vol:.4f}, Beta: {beta:.4f}, VaR: {var:.4f}, CVaR: {cvar:.4f}, Treynor: {treynor:.4f}")
else:
    print("\nSkipping Risk-Free Rate Sensitivity Analysis (not applicable for selected objective)")


In [ ]:
# Compute optimal_port_returns
try:
    optimal_port_returns = log_returns_subset @ optimal_weights
    if not np.isfinite(optimal_port_returns).all():
        print("Warning: Non-finite values in optimal_port_returns, replacing with 0")
        optimal_port_returns = optimal_port_returns.where(np.isfinite(optimal_port_returns), 0)
except Exception as e:
    print(f"Error in computing optimal_port_returns: {e}")
    optimal_port_returns = pd.Series(0, index=log_returns_subset.index)

# Statistical Tests
try:
    equal_port_returns = log_returns_subset @ equal_weights
    if len(optimal_port_returns) != len(equal_port_returns) or len(optimal_port_returns) != len(market_returns):
        print("Warning: Mismatched lengths for statistical tests")
    else:
        _, p_val_norm_opt = shapiro(optimal_port_returns)
        _, p_val_norm_eq = shapiro(equal_port_returns)
        print(f"\nNormality Test (Shapiro-Wilk): Optimal p={p_val_norm_opt:.4f}, Equal-Weight p={p_val_norm_eq:.4f}")
        if not np.allclose(optimal_port_returns, equal_port_returns):
            t_stat, p_val = ttest_rel(optimal_port_returns, equal_port_returns)
            print(f"Paired t-test (Optimal vs Equal-Weight): T={t_stat:.4f}, p={p_val:.4f}")
            wilcoxon_stat, p_val_w = wilcoxon(optimal_port_returns, equal_port_returns, zero_method='zsplit')
            print(f"Wilcoxon Test (Optimal vs Equal-Weight): Stat={wilcoxon_stat:.4f}, p={p_val_w:.4f}")
        else:
            print("Skipping paired t-test and Wilcoxon test: Optimal and Equal-Weight portfolios are identical")
        f_stat, p_val_f = f_oneway(optimal_port_returns, equal_port_returns, market_returns)
        print(f"F-test (Optimal vs Equal-Weight vs Market): F={f_stat:.4f}, p={p_val_f:.4f}")
        t_stat_market, p_val_market = ttest_ind(optimal_port_returns, market_returns)
        print(f"Independent t-test (Optimal vs Market): T={t_stat_market:.4f}, p={p_val_market:.4f}")
except Exception as e:
    print(f"Error in statistical tests: {e}")


In [ ]:
# Portfolio Rebalancing with Transaction Costs
transaction_cost = 0.001
rebal_period = 252
rebal_weights = equal_weights.copy()
rebal_port_returns = pd.Series(index=log_returns_subset.index, dtype=float)
total_transaction_costs = 0
previous_weights = rebal_weights.copy()
print("\nRebalancing Diagnostics:")
for i in range(0, len(log_returns_subset), rebal_period):
    period_returns = log_returns_subset.iloc[i:i+rebal_period]
    period_index = period_returns.index
    print(f"Period {i//rebal_period + 1}: {period_index[0]} to {period_index[-1] if len(period_index) > 1 else period_index[0]}, Length: {len(period_returns)}")
    if len(period_returns) >= 30:
        try:
            period_cov = LedoitWolf().fit(period_returns).covariance_ * 252
            cond_number = np.linalg.cond(period_cov)
            if cond_number > 1e6:
                print("  Warning: High condition number, applying regularization")
                period_cov += np.eye(period_cov.shape[0]) * 1e-4
            if not np.any(np.isnan(period_cov)):
                rebal_constraints = [
                    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
                    {'type': 'ineq', 'fun': lambda weights: safe_vol_constraint(weights, max_vol=safe_equal_vol * 1.1)},
                ]
                for asset_class, positions in asset_class_indices.items():
                    target_weight = target_exposures[asset_class]
                    rebal_constraints.append(
                        {'type': 'eq', 'fun': lambda w, pos=positions, tw=target_weight: safe_asset_class_constraint(w, pos, tw)}
                    )
                opt_res = minimize(
                    objective_fun,
                    rebal_weights,
                    args=(period_returns, period_cov, risk_free_rate, asset_market_cov, market_var),
                    method='SLSQP',
                    constraints=rebal_constraints,
                    bounds=bounds,
                    options={'disp': False, 'maxiter': 2000, 'ftol': 1e-9, 'eps': 1e-8}
                )
                if opt_res.success:
                    new_weights = opt_res.x
                    weight_changes = np.abs(new_weights - previous_weights)
                    period_transaction_cost = transaction_cost * weight_changes.sum()
                    total_transaction_costs += period_transaction_cost
                    rebal_weights = new_weights
                    previous_weights = rebal_weights.copy()
                    print(f"  Optimization successful, new weights sum: {np.sum(rebal_weights):.4f}, Transaction cost: {period_transaction_cost:.6f}")
                else:
                    print(f"  Optimization failed: {opt_res.message}, using previous weights")
                    rebal_weights = previous_weights
            else:
                print("  Invalid covariance matrix, using previous weights")
                rebal_weights = previous_weights
        except Exception as e:
            print(f"  Error in rebalancing optimization: {e}, using previous weights")
            rebal_weights = previous_weights
    else:
        print("  Insufficient data, using previous weights")
        rebal_weights = previous_weights
    period_port_returns = period_returns @ rebal_weights
    if i > 0 and 'period_transaction_cost' in locals() and period_transaction_cost > 0:
        period_port_returns.iloc[0] -= period_transaction_cost
    rebal_port_returns[period_index] = period_port_returns
    drift = np.sum(np.abs(rebal_weights - initial_weights))
    print(f"  Portfolio drift from initial weights: {drift:.4f}")

print("Length of rebal_port_returns:", len(rebal_port_returns))
print("Any NaN in rebal_port_returns?", rebal_port_returns.isna().sum())
print(f"Total Transaction Costs: {total_transaction_costs:.6f}")


In [ ]:
# Visualization: Weights
try:
    non_zero_indices = optimal_weights > 1e-6
    non_zero_weights = optimal_weights[non_zero_indices]
    non_zero_labels = log_returns_subset.columns[non_zero_indices]
    sorted_indices = np.argsort(non_zero_weights)[::-1]
    sorted_weights = non_zero_weights[sorted_indices]
    sorted_labels = non_zero_labels[sorted_indices]

    plt.style.use('seaborn')
    plt.figure(figsize=(12, 6))
    bars = plt.bar(range(len(sorted_labels)), sorted_weights)
    plt.xlabel('Assets', fontsize=12)
    plt.ylabel('Weights', fontsize=12)
    plt.title(f'Optimal Portfolio Weights for {selected_objective} with {selected_constraint} (Non-Zero, Sorted Descending)', fontsize=14)
    plt.xticks(range(len(sorted_labels)), sorted_labels, rotation=45, ha='right')
    for bar, weight in zip(bars, sorted_weights):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{weight:.3f}',
                 ha='center', va='bottom')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 8))
    plt.pie(sorted_weights, labels=sorted_labels, autopct='%1.1f%%', startangle=90)
    plt.title(f'Optimal Portfolio Weights for {selected_objective} with {selected_constraint} (Pie Chart, Non-Zero, Sorted Descending)', fontsize=14)
    plt.axis('equal')
    plt.show()
except Exception as e:
    print(f"Error in weights visualization: {e}")


In [ ]:
# Visualization: Efficient Frontier
try:
    returns_range = np.linspace(0, 0.5, 50)
    volatilities = []
    for ret in returns_range:
        constraints = [
            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
            {'type': 'eq', 'fun': lambda weights: calc_expected_return(weights, log_returns_subset) - ret}
        ]
        for asset_class, positions in asset_class_indices.items():
            target_weight = target_exposures[asset_class]
            constraints.append(
                {'type': 'eq', 'fun': lambda w, pos=positions, tw=target_weight: safe_asset_class_constraint(w, pos, tw)}
            )
        opt_res = minimize(
            calc_standard_deviation,
            initial_weights_perturbed,
            args=(port_cov_matrix,),
            method='SLSQP',
            constraints=constraints,
            bounds=bounds,
            options={'maxiter': 2000, 'ftol': 1e-9, 'eps': 1e-8}
        )
        volatilities.append(opt_res.fun if opt_res.success else np.nan)

    valid_idx = ~np.isnan(volatilities)
    returns_range = returns_range[valid_idx]
    volatilities = np.array(volatilities)[valid_idx]

    plt.figure(figsize=(10, 6))
    plt.scatter(volatilities, returns_range, label='Efficient Frontier', color='blue', alpha=0.6)
    plt.scatter(opt_vol, opt_return, color='red', marker='*', s=200, label='Optimal Portfolio')
    plt.xlabel('Volatility', fontsize=12)
    plt.ylabel('Expected Return', fontsize=12)
    plt.title(f'Efficient Frontier for {selected_objective} with {selected_constraint}', fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Error in efficient frontier visualization: {e}")

In [ ]:
# Visualization: Correlation Heatmap
try:
    plt.figure(figsize=(10, 8))
    sns.heatmap(log_returns_subset.corr(), annot=False, cmap='coolwarm')
    plt.title('Correlation Heatmap of Asset Returns', fontsize=14)
    plt.show()
except Exception as e:
    print(f"Error in correlation heatmap: {e}")


In [ ]:
# Visualization: Returns Distribution
try:
    plt.figure(figsize=(10, 6))
    plt.hist(optimal_port_returns, bins=50, alpha=0.7, label='Optimal Portfolio')
    plt.hist(market_returns, bins=50, alpha=0.7, label='Market (Sensex)')
    plt.xlabel('Daily Returns', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.title(f'Returns Distribution for {selected_objective} with {selected_constraint}', fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.show()
except Exception as e:
    print(f"Error in returns distribution visualization: {e}")


In [ ]:
# Visualization: Rebalanced vs Static Portfolio Cumulative Returns
try:
    plt.figure(figsize=(12, 6))
    plt.plot(log_returns_subset.index, np.exp(np.cumsum(optimal_port_returns)) - 1, label='Static Optimal')
    plt.plot(rebal_port_returns.index, np.exp(np.cumsum(rebal_port_returns.dropna())) - 1, label='Rebalanced')
    plt.plot(market_returns.index, np.exp(np.cumsum(market_returns)) - 1, label='Market (Sensex)')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Cumulative Returns', fontsize=12)
    plt.title(f'Cumulative Returns: Rebalanced vs Static vs Market for {selected_objective} with {selected_constraint}', fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.gca().xaxis.set_major_locator(YearLocator())
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Error in cumulative returns visualization: {e}")


In [ ]:
# Visualization: Drawdown Plot
try:
    cum_returns = np.exp(np.cumsum(optimal_port_returns)) - 1
    rolling_max = np.maximum.accumulate(cum_returns + 1)
    drawdowns = (rolling_max - (cum_returns + 1)) / rolling_max
    max_dd = drawdowns.max() if np.isfinite(drawdowns).all() else np.nan
    t_stat_dd, p_val_dd = ttest_1samp(drawdowns, 0) if np.isfinite(drawdowns).all() else (np.nan, np.nan)
    print(f"\nMax Drawdown: {max_dd:.4f}")
    print(f"Drawdown t-test: T={t_stat_dd:.4f}, p={p_val_dd:.4f}")

    drawdowns_series = pd.Series(drawdowns, index=optimal_port_returns.index)
    plt.figure(figsize=(12, 6))
    plt.plot(drawdowns_series, label='Drawdowns', color='red')
    plt.axhline(y=max_dd, color='black', linestyle='dashed', label=f'Max Drawdown: {max_dd:.4f}')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Drawdown', fontsize=12)
    plt.title(f'Drawdowns of Optimal Portfolio for {selected_objective} with {selected_constraint}', fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.gca().xaxis.set_major_locator(YearLocator())
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Error in drawdown visualization: {e}")


In [ ]:
# Monte Carlo Simulation
try:
    num_simulations = [500, 1000, 2000]
    for n_sim in num_simulations:
        df_t = 3
        sim_returns = t.rvs(df_t, loc=log_returns_subset.mean() * 252, scale=np.sqrt(np.diag(port_cov_matrix)), size=(n_sim, num_assets))
        sim_port_returns = sim_returns @ optimal_weights
        plt.figure(figsize=(10, 6))
        plt.hist(sim_port_returns, bins=50, alpha=0.7, label=f'Monte Carlo (n={n_sim})')
        plt.axvline(opt_return, color='red', linestyle='dashed', label=f'Expected Return: {opt_return:.4f}')
        plt.xlabel('Annualized Return', fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        plt.title(f'Monte Carlo Simulation of Portfolio Returns (n={n_sim}) for {selected_objective} with {selected_constraint}', fontsize=14)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        print(f"Monte Carlo (n={n_sim}) Mean Return: {np.mean(sim_port_returns):.4f}, Std: {np.std(sim_port_returns):.4f}")
except Exception as e:
    print(f"Error in Monte Carlo simulation: {e}")


In [ ]:
# Stress Testing
try:
    print("\nStress Testing:")
    scenarios = {
        'Severe Market Crash': {'shock': -0.50, 'type': 'return'},
        'Mild Recession': {'shock': -0.15, 'type': 'return'},
        'Economic Boom': {'shock': 0.20, 'type': 'return'},
        'Increased Volatility': {'shock': 2.0, 'type': 'vol'},
    }

    stress_results = {}
    for name, params in scenarios.items():
        if params['type'] == 'return':
            shock = params['shock']
            stressed_port_return = opt_return + opt_beta * shock
            stress_results[name] = stressed_port_return
            print(f"{name}: Stressed Annual Return: {stressed_port_return:.4f}")
        elif params['type'] == 'vol':
            shock = params['shock']
            stressed_vol = opt_vol * shock
            stressed_sharpe = (opt_return - risk_free_rate) / stressed_vol if np.isfinite(stressed_vol) and stressed_vol != 0 else np.nan
            stress_results[name] = (stressed_vol, stressed_sharpe)
            print(f"{name}: Stressed Volatility: {stressed_vol:.4f}, Stressed Sharpe: {stressed_sharpe:.4f}")
except Exception as e:
    print(f"Error in stress testing: {e}")

# Stress Monte Carlo
try:
    stress_mean = log_returns_subset.mean() * 252 - 0.10
    stress_cov = port_cov_matrix * 4
    stressed_sim_returns = np.random.multivariate_normal(stress_mean, stress_cov, num_simulations[-1])
    stressed_port_returns = stressed_sim_returns @ optimal_weights
    mean_stressed_return = np.mean(stressed_port_returns)
    var_stressed = np.percentile(stressed_port_returns, 5) if np.isfinite(stressed_port_returns).all() else np.nan
    print(f"Stressed Monte Carlo Mean Return: {mean_stressed_return:.4f}")
    print(f"Stressed VaR (5%): {var_stressed:.4f}")

    simulated_port_returns = np.random.multivariate_normal(log_returns_subset.mean() * 252, port_cov_matrix, num_simulations[-1]) @ optimal_weights
    plt.figure(figsize=(10, 6))
    plt.hist(simulated_port_returns, bins=50, alpha=0.7, label='Normal')
    plt.hist(stressed_port_returns, bins=50, alpha=0.7, label='Stressed', color='red')
    plt.axvline(opt_return, color='blue', linestyle='dashed', label='Normal Expected')
    plt.axvline(mean_stressed_return, color='black', linestyle='dashed', label='Stressed Expected')
    plt.xlabel('Annualized Return', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.title(f'Normal vs Stressed Monte Carlo Simulations for {selected_objective} with {selected_constraint}', fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    t_stat_stress, p_val_stress = ttest_ind(simulated_port_returns, stressed_port_returns)
    print(f"t-test Normal vs Stressed Returns: T={t_stat_stress:.4f}, p={p_val_stress:.4f}")
except Exception as e:
    print(f"Error in stress Monte Carlo: {e}")


In [ ]:
# Forecasting
def check_stationarity(series):
    try:
        result = adfuller(series.dropna(), autolag='AIC')
        print(f"ADF Statistic: {result[0]:.4f}, p-value: {result[1]:.4f}")
        return result[1] < 0.05
    except Exception as e:
        print(f"Error in stationarity check: {e}")
        return False

def create_lagged_features(series, lags=5):
    try:
        df = pd.DataFrame(series, columns=['returns'])
        for i in range(1, lags + 1):
            df[f'lag_{i}'] = series.shift(i)
        return df.dropna()
    except Exception as e:
        print(f"Error in create_lagged_features: {e}")
        return pd.DataFrame()

def evaluate_forecast(test, forecast, model_name, train, rmse):
    try:
        forecast = pd.Series(forecast, index=test.index, name='forecast')
        print(f"\n{model_name} Forecast Diagnostics:")
        print("NaN in forecast:", np.isnan(forecast).sum())
        print("Inf in forecast:", np.isinf(forecast).sum())
        print("Forecast index first/last:", forecast.index[0], forecast.index[-1])
        print("Index match with test?", forecast.index.equals(test.index))
        print("Forecast first 5 values:", forecast[:5].values)

        if not forecast.index.equals(test.index):
            print(f"Warning: Index mismatch in {model_name}. Reindexing forecast.")
            forecast = forecast.reindex(test.index, method='nearest')

        forecast_errors = forecast - test
        print("NaN in forecast_errors:", forecast_errors.isna().sum())
        print("Inf in forecast_errors:", np.isinf(forecast_errors).sum())
        print("Forecast_errors first 5 values:", forecast_errors.head().values)
        print(f"{model_name} RMSE: {rmse:.4f}")

        plt.figure(figsize=(12, 6))
        plt.plot(train.index, train, label='Train')
        plt.plot(test.index, test, label='Test')
        plt.plot(test.index, forecast, label=f'{model_name} Forecast', linestyle='--')
        plt.xlabel('Date', fontsize=12)
        plt.ylabel('Portfolio Returns', fontsize=12)
        plt.title(f'Portfolio Returns Forecast with {model_name}', fontsize=14)
        plt.legend()
        plt.grid(True)
        plt.gca().xaxis.set_major_locator(YearLocator())
        plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

        valid_errors = forecast_errors.dropna()
        if len(valid_errors) > 0 and np.all(np.isfinite(valid_errors)):
            t_stat, p_val = ttest_1samp(valid_errors, 0)
            print(f"{model_name} Forecast t-test: T={t_stat:.4f}, p={p_val:.4f}")
            plt.figure(figsize=(10, 6))
            plt.hist(valid_errors, bins=30, alpha=0.7, edgecolor='black', label=f'{model_name} Forecast Errors')
            plt.axvline(0, color='red', linestyle='dashed', label='Zero Error')
            plt.xlabel('Forecast Error', fontsize=12)
            plt.ylabel('Frequency', fontsize=12)
            plt.title(f'Distribution of {model_name} Forecast Errors', fontsize=14)
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()
        else:
            print(f"Warning: Cannot perform t-test for {model_name}. Valid errors length: {len(valid_errors)}")
        return forecast, valid_errors
    except Exception as e:
        print(f"Error in evaluate_forecast for {model_name}: {e}")
        return pd.Series(), pd.Series()

# Forecasting Setup
try:
    port_returns_ts = pd.Series(optimal_port_returns, index=log_returns.index[:len(optimal_port_returns)])
    print("\nForecasting Diagnostics:")
    print("Length of port_returns_ts:", len(port_returns_ts))
    print("NaN in port_returns_ts:", port_returns_ts.isna().sum())
    print("Inf in port_returns_ts:", np.isinf(port_returns_ts).sum())
    print("Std of port_returns_ts:", port_returns_ts.std())

    stationary = check_stationarity(port_returns_ts)
    if not stationary:
        print("Series is non-stationary. Applying first differencing.")
        port_returns_ts_diff = port_returns_ts.diff().dropna()
        print("Stationarity check on differenced series:")
        stationary = check_stationarity(port_returns_ts_diff)
        if stationary:
            port_returns_ts = port_returns_ts_diff
            print("Using differenced series for forecasting.")
        else:
            print("Warning: Differenced series is still non-stationary. Proceed with caution.")

    port_returns_ts = port_returns_ts.dropna()
    train_size = int(len(port_returns_ts) * 0.8)
    train = port_returns_ts[:train_size]
    test = port_returns_ts[train_size:]
    print("Train length:", len(train))
    print("Test length:", len(test))
    print("Train index first/last:", train.index[0], train.index[-1])
    print("Test index first/last:", test.index[0], test.index[-1])
    print("Train first 5 values:", train.head().values)
    print("Test first 5 values:", test.head().values)

    forecasts = {}
    valid_errors_dict = {}

    # Baseline: Moving Average
    ma_window = 20
    forecast_ma = train.rolling(window=ma_window).mean().iloc[-1]
    forecast_ma = pd.Series([forecast_ma] * len(test), index=test.index)
    rmse_ma = np.sqrt(mean_squared_error(test, forecast_ma)) if np.isfinite(test).all() and np.isfinite(forecast_ma).all() else np.nan
    forecast_ma, errors_ma = evaluate_forecast(test, forecast_ma, "Moving Average", train, rmse_ma)
    forecasts['Moving Average'] = forecast_ma
    valid_errors_dict['Moving Average'] = errors_ma

    # ARIMA
    try:
        model_arima = ARIMA(train.values, order=(1,0,0) if stationary else (1,1,1), trend='c', enforce_stationarity=True, enforce_invertibility=True)
        result_arima = model_arima.fit(method_kwargs={'maxiter': 200})
        forecast_arima = result_arima.get_forecast(steps=len(test)).predicted_mean
        forecast_arima = pd.Series(forecast_arima, index=test.index)
        if np.any(np.isnan(forecast_arima)) or np.any(np.isinf(forecast_arima)):
            print("Warning: ARIMA forecast contains NaN or Inf. Using moving average.")
            forecast_arima = forecast_ma
        else:
            rmse_arima = np.sqrt(mean_squared_error(test, forecast_arima)) if np.isfinite(test).all() and np.isfinite(forecast_arima).all() else np.nan
            forecast_arima, errors_arima = evaluate_forecast(test, forecast_arima, "ARIMA", train, rmse_arima)
            forecasts['ARIMA'] = forecast_arima
            valid_errors_dict['ARIMA'] = errors_arima
    except Exception as e:
        print(f"Error in ARIMA fitting: {e}")
        print("Using moving average as fallback for ARIMA")
        forecast_arima = forecast_ma
        rmse_arima = rmse_ma
        forecast_arima, errors_arima = evaluate_forecast(test, forecast_arima, "ARIMA", train, rmse_arima)
        forecasts['ARIMA'] = forecast_arima
        valid_errors_dict['ARIMA'] = errors_arima

    # Auto-ARIMA
    if PMDARIMA_AVAILABLE:
        try:
            model_auto_arima = auto_arima(
                train.values,
                seasonal=False,
                suppress_warnings=True,
                maxiter=200,
                start_p=0,
                start_q=0,
                max_p=3,
                max_q=3,
                max_d=3,
                method='nm',
                error_action='ignore',
                trace=True
            )
            forecast_auto_arima = model_auto_arima.predict(n_periods=len(test))
            forecast_auto_arima = pd.Series(forecast_auto_arima, index=test.index)
            if np.any(np.isnan(forecast_auto_arima)) or np.any(np.isinf(forecast_auto_arima)):
                print("Warning: Auto-ARIMA forecast contains NaN or Inf. Using moving average.")
                forecast_auto_arima = forecast_ma
            else:
                rmse_auto_arima = np.sqrt(mean_squared_error(test, forecast_auto_arima)) if np.isfinite(test).all() and np.isfinite(forecast_auto_arima).all() else np.nan
                forecast_auto_arima, errors_auto_arima = evaluate_forecast(test, forecast_auto_arima, "Auto-ARIMA", train, rmse_auto_arima)
                forecasts['Auto-ARIMA'] = forecast_auto_arima
                valid_errors_dict['Auto-ARIMA'] = errors_auto_arima
        except Exception as e:
            print(f"Error in Auto-ARIMA fitting: {e}")
            print("Using moving average as fallback for Auto-ARIMA")
            forecast_auto_arima = forecast_ma
            rmse_auto_arima = rmse_ma
            forecast_auto_arima, errors_auto_arima = evaluate_forecast(test, forecast_auto_arima, "Auto-ARIMA", train, rmse_auto_arima)
            forecasts['Auto-ARIMA'] = forecast_auto_arima
            valid_errors_dict['Auto-ARIMA'] = errors_auto_arima
    else:
        print("Auto-ARIMA skipped (pmdarima not available). Using ARIMA as fallback.")
        forecasts['Auto-ARIMA'] = forecast_arima
        valid_errors_dict['Auto-ARIMA'] = errors_arima

    # XGBoost
    if XGBOOST_AVAILABLE:
        try:
            lags = 5
            train_lagged = create_lagged_features(port_returns_ts, lags)
            train_lagged = train_lagged[train_lagged.index <= train.index[-1]]
            test_lagged = create_lagged_features(port_returns_ts, lags)
            test_lagged = test_lagged[test_lagged.index >= test.index[0]]
            X_train = train_lagged.drop(columns='returns')
            y_train = train_lagged['returns']
            X_test = test_lagged.drop(columns='returns')
            y_test = test_lagged['returns']
            model_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
            model_xgb.fit(X_train, y_train)
            forecast_xgb = model_xgb.predict(X_test)
            forecast_xgb = pd.Series(forecast_xgb, index=y_test.index)
            rmse_xgb = np.sqrt(mean_squared_error(y_test, forecast_xgb)) if np.isfinite(y_test).all() and np.isfinite(forecast_xgb).all() else np.nan
            forecast_xgb, errors_xgb = evaluate_forecast(test.reindex(y_test.index), forecast_xgb, "XGBoost", train, rmse_xgb)
            forecasts['XGBoost'] = forecast_xgb
            valid_errors_dict['XGBoost'] = errors_xgb
        except Exception as e:
            print(f"Error in XGBoost fitting: {e}")

    # LSTM
    if TENSORFLOW_AVAILABLE:
        try:
            lags = 5
            scaler = MinMaxScaler()
            train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
            test_scaled = scaler.transform(test.values.reshape(-1, 1))
            def create_sequences(data, seq_length):
                X, y = [], []
                for i in range(len(data) - seq_length):
                    X.append(data[i:i+seq_length])
                    y.append(data[i+seq_length])
                return np.array(X), np.array(y)
            X_train, y_train = create_sequences(train_scaled, lags)
            X_test, y_test = create_sequences(test_scaled, lags)
            model_lstm = Sequential([
                LSTM(50, activation='relu', input_shape=(lags, 1), return_sequences=False),
                Dense(1)
            ])
            model_lstm.compile(optimizer='adam', loss='mse')
            model_lstm.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)
            forecast_lstm = []
            current_sequence = test_scaled[-lags:].reshape(1, lags, 1)
            for _ in range(len(test)):
                pred = model_lstm.predict(current_sequence, verbose=0)
                forecast_lstm.append(pred[0, 0])
                current_sequence = np.roll(current_sequence, -1, axis=1)
                current_sequence[0, -1, 0] = pred[0, 0]
            forecast_lstm = scaler.inverse_transform(np.array(forecast_lstm).reshape(-1, 1)).flatten()
            forecast_lstm = pd.Series(forecast_lstm, index=test.index)
            rmse_lstm = np.sqrt(mean_squared_error(test, forecast_lstm)) if np.isfinite(test).all() and np.isfinite(forecast_lstm).all() else np.nan
            forecast_lstm, errors_lstm = evaluate_forecast(test, forecast_lstm, "LSTM", train, rmse_lstm)
            forecasts['LSTM'] = forecast_lstm
            valid_errors_dict['LSTM'] = errors_lstm
        except Exception as e:
            print(f"Error in LSTM fitting: {e}")

    # Ensemble
    try:
        valid_forecasts = {k: v for k, v in forecasts.items() if not np.any(np.isnan(v))}
        if valid_forecasts:
            ensemble_forecast = pd.DataFrame(valid_forecasts).mean(axis=1)
            rmse_ensemble = np.sqrt(mean_squared_error(test.reindex(ensemble_forecast.index), ensemble_forecast)) if np.isfinite(test).all() and np.isfinite(ensemble_forecast).all() else np.nan
            ensemble_forecast, errors_ensemble = evaluate_forecast(test.reindex(ensemble_forecast.index), ensemble_forecast, "Ensemble", train, rmse_ensemble)
            forecasts['Ensemble'] = ensemble_forecast
            valid_errors_dict['Ensemble'] = errors_ensemble
        else:
            print("No valid forecasts for Ensemble.")
    except Exception as e:
        print(f"Error in Ensemble fitting: {e}")

    # RMSE Comparison
    print("\nRMSE Comparison:")
    for model_name, forecast in forecasts.items():
        if not np.any(np.isnan(forecast)) and np.isfinite(test).all() and np.isfinite(forecast).all():
            rmse = np.sqrt(mean_squared_error(test.reindex(forecast.index), forecast))
            print(f"{model_name}: {rmse:.4f}")
        else:
            print(f"{model_name}: Skipped due to NaN or Inf in forecast")
except Exception as e:
    print(f"Error in forecasting section: {e}")